In [1]:
sc.install_pypi_package("boto3")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1675699948280_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/52/34/1f9331d37ec16e375ef8a5da328b2a7cef1b1a327a1800f3904f2420f0f3/boto3-1.26.64-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6c/dc/53ea6594209c1b38f4aa61a413f82d78c008ecab3a45dbf0303902206768/botocore-1.29.64-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5e/c6/af903b5fab3f9b5b1e883f49a770066314c6dcceb589cf938d48c89556c1/s3transfer-0.6.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fe/ca/466766e20b767ddb9b951202542310cba37ea5f2d792dae7589f1741af58/urllib3-1.26.14-py2.py3-none-any.whl

You are using pip version 9.0.1, however version 23.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [2]:
import os
import boto3
import json

os.environ[
    'PYSPARK_SUBMIT_ARGS'] = '--packages=com.qubole.spark/spark-sql-kinesis_2.11/1.1.3-spark_2.4 pyspark-shell'


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
stream_name = 'pyspark-kinesis'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
kinesis_client = boto3.client('kinesis', region_name='us-east-1')


response = kinesis_client.describe_stream(StreamName=stream_name)
print(response)
shard_id = response['StreamDescription']['Shards'][0]['ShardId']



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'StreamDescription': {'StreamName': 'pyspark-kinesis', 'StreamARN': 'arn:aws:kinesis:us-east-1:198423591225:stream/pyspark-kinesis', 'StreamStatus': 'ACTIVE', 'StreamModeDetails': {'StreamMode': 'PROVISIONED'}, 'Shards': [{'ShardId': 'shardId-000000000000', 'HashKeyRange': {'StartingHashKey': '0', 'EndingHashKey': '340282366920938463463374607431768211455'}, 'SequenceNumberRange': {'StartingSequenceNumber': '49637767260392005847003959566510704681299211336502214658'}}], 'HasMoreShards': False, 'RetentionPeriodHours': 24, 'StreamCreationTimestamp': datetime.datetime(2023, 2, 6, 16, 23, 20, tzinfo=tzlocal()), 'EnhancedMonitoring': [{'ShardLevelMetrics': []}], 'EncryptionType': 'NONE'}, 'ResponseMetadata': {'RequestId': 'fbfce293-dea4-1022-a7a7-cee45cc7e733', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'fbfce293-dea4-1022-a7a7-cee45cc7e733', 'x-amz-id-2': '0MuE4ywoWEH0txlXM1kFYqwWw/P5sth4+KmVIxr88iDW2xdyJSIrdRFoBB/v3Aws960E5LMgCPI9RV8IrbUgvlkefvLo5NIB', 'date': 'Mon, 06 Feb 

In [6]:

response = kinesis_client.get_shard_iterator(StreamName=stream_name,
                                             ShardId=shard_id,
                                             ShardIteratorType='TRIM_HORIZON')
print(response)
shard_iterator = response['ShardIterator']


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ShardIterator': 'AAAAAAAAAAHJt3h+sBZxwp7Ni2YGtMu4lDP7OSD9xMWep1mQavrnbSzKDpTkIKqIgRePc9Rf3tZDd0eXc1HEBuZTXQiOHczTpLtr6BiEClJsgJJByL77TzJPr9WUrletsnB45g0kDzn0813TpCi51Ugy+hx1VbMMwHz7tsZ17E9jE/apDEnrtIMA6RPv22PGrsFa9Lb4QjntwpYebnm1QvztAbETB2/kQA5KL6BMUWZ7Pz7lZQtkIw==', 'ResponseMetadata': {'RequestId': 'c0df1de0-8541-28a1-9c84-31990722dfb0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c0df1de0-8541-28a1-9c84-31990722dfb0', 'x-amz-id-2': 'lMmvGJFMIj79+6LqwrN9H5IjaJjfiRBRzhdpWxaRo6iKlw87sDJ/53ubN6U3UgpixPoP/x0HDNOWLNW4dHMprtx5quEbAahy', 'date': 'Mon, 06 Feb 2023 16:26:33 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '268'}, 'RetryAttempts': 0}}

In [7]:
#-----------------------------------------------------------------
# Get the records.
# Get max_records from the shard, or run continuously if you wish.
#-----------------------------------------------------------------

max_records = 30
record_count = 0

while record_count < max_records:

    response = kinesis_client.get_records(ShardIterator=shard_iterator,
                                          Limit=4)
    # print(response)
    shard_iterator = response['NextShardIterator']
    records = response['Records']
    record_count += len(records)
    # process_records(records)

    # print(records[1])
    if len(records) > 0:
        for i in range(len(records)):
            ms1 = json.loads(records[i]['Data'].decode('utf-8'))
            print(ms1)
    #     print(ms1['count'])


        

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'message_type': 'message1', 'count': 2}
{'message_type': 'message2', 'count': 1}
{'message_type': 'message1', 'count': 2}
{'message_type': 'message3', 'count': 3}
{'message_type': 'message1', 'count': 5}
{'message_type': 'message1', 'count': 2}
{'message_type': 'message2', 'count': 1}
{'message_type': 'message1', 'count': 2}
{'message_type': 'message3', 'count': 3}
{'message_type': 'message1', 'count': 5}
{'message_type': 'message1', 'count': 2}
{'message_type': 'message2', 'count': 1}
{'message_type': 'message1', 'count': 2}
{'message_type': 'message3', 'count': 3}
{'message_type': 'message1', 'count': 5}
{'message_type': 'message1', 'count': 2}
{'message_type': 'message2', 'count': 1}
{'message_type': 'message1', 'count': 2}
{'message_type': 'message3', 'count': 3}
{'message_type': 'message1', 'count': 5}
{'message_type': 'message1', 'count': 2}
{'message_type': 'message2', 'count': 1}
{'message_type': 'message1', 'count': 2}
{'message_type': 'message3', 'count': 3}
{'message_type':

In [8]:
import time
for i in range(5):
    time.sleep(1)
    print(i)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0
1
2
3
4

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…